In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import collections
warnings.filterwarnings("ignore")
from wordcloud import WordCloud,STOPWORDS 
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from collections import Counter,OrderedDict
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
import pickle
from scipy.stats import norm
from math import sqrt
from plotly.colors import n_colors
from plotly.subplots import make_subplots
from IPython.display import Image
from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [ ]:
#l oad dataset
movies_data = pd.read_csv('../input/movies-on-netflix-prime-video-hulu-and-disney/MoviesOnStreamingPlatforms_updated.csv')

In [ ]:
#l oad dataset
movies_datas = pd.read_csv('../input/movies-on-netflix-prime-video-hulu-and-disney/MoviesOnStreamingPlatforms_updated.csv')

In [ ]:
#Understanding the data
movies_datas.head(25)

In [ ]:
movies_datas.columns

In [ ]:
movies_datas.describe()

****Dropping unncessary columns
****

In [ ]:
movies_datas.drop(['Unnamed: 0'], axis = 1,inplace=True) 

In [ ]:
movies_datas.isnull().sum()

# ****Filling the missingvalue with the mean

In [ ]:
movies_datas.fillna(movies_datas.mean(axis=0),inplace=True)
movies_datas

In [ ]:
movies_datas.isnull().sum()

In [ ]:
movies_data.mean()

In [ ]:
movies_data.update(movies_data[['Age']].fillna('all'))
movies_data.update(movies_data[['Directors', 'Genres', 'Country', 'Language']].fillna('Others'))
movies_data.update(movies_data[['Runtime']].fillna(93))
movies_data.update(movies_data[['IMDb']].fillna(5.91))
movies_data.update(movies_data[['Rotten Tomatoes']].fillna('50%'))
movies_data

In [ ]:
movies_data.isna().sum()


In [ ]:
pandas_profiling.ProfileReport(movies_data)

# Data Analysis

In [ ]:
#Total number of unique directors
len(movies_data['Directors'].unique())

In [ ]:
len(movies_data['Genres'].unique())

In [ ]:
len(movies_data['Language'].unique())

In [ ]:
print('Minimum IMDb rating: ', movies_data['IMDb'].min())
print('Maximum IMDb rating: ', movies_data['IMDb'].max())

In [ ]:
indian_movies = movies_datas[movies_datas.Country =='India']
indian_movies

In [ ]:
print('Total number of Indian movies                : ',indian_movies['Title'].count() )


In [ ]:
telugu_movies = movies_data[movies_data.Language =='Telugu']
telugu_movies

In [ ]:
print('Total number of Telugu movies                : ',telugu_movies['Title'].count() )


In [ ]:
netflix_movies = movies_datas[movies_datas['Netflix'] == 1]
hulu_movies = movies_datas[movies_datas['Hulu'] == 1]
prime_movies =movies_datas[movies_datas['Prime Video'] == 1]
disney_movies = movies_datas[movies_datas['Disney+'] == 1]

In [ ]:
def generate_word_cloud(text):
    wordcloud = WordCloud(
        width = 3000,
        height = 2000,
        background_color = 'black').generate(str(text))
    fig = plt.figure(
        figsize = (40, 30),
        facecolor = 'k',
        edgecolor = 'k')
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.show()

In [ ]:
netflix_titles = netflix_movies.Title[:500].values
generate_word_cloud(netflix_titles)

In [ ]:
prime_titles = prime_movies.Title[:500].values
generate_word_cloud(prime_titles)

In [ ]:
hulu_titles = hulu_movies.Title[:500].values
generate_word_cloud(hulu_titles)

# visualising the top 10 longest movies according to runtime

In [ ]:
top_Runtime = movies_datas.sort_values('Runtime',ascending = False).head(20)

In [ ]:
fig = px.bar(top_Runtime, x='Title', y='Runtime', color='Runtime', height=500, title='Runtime of the top 10 longest movies')
fig.show()

In [ ]:
## Movie count Analysis

In [ ]:
yearly_movie_count = movies_data.groupby('Year')['Title'].count().reset_index().rename(columns = {'Title':'MovieCount'})

In [ ]:
yearly_movie_count = yearly_movie_count.sort_values(by='MovieCount',ascending=False)

In [ ]:
yearly_movie_count.head(30)

In [ ]:
fig = px.bar(yearly_movie_count[:30], x='Year', y='MovieCount', color='MovieCount', height=600)
fig.show()

In [ ]:
netflix_movies_count = len(movies_datas[movies_datas['Netflix'] == 1].index)
hulu_movies_count = len(movies_datas[movies_datas['Hulu'] == 1].index)
prime_movies_count =len(movies_datas[movies_datas['Prime Video'] == 1].index)
disney_movies_count = len(movies_datas[movies_datas['Disney+'] == 1].index)


print(netflix_movies_count)
print(hulu_movies_count)
print(prime_movies_count)
print(disney_movies_count)

In [ ]:
label=['Netflix','Hulu', 'Prime Video','Disney+']
count=[netflix_movies_count,hulu_movies_count,prime_movies_count,disney_movies_count]
platform = pd.DataFrame(
    {'Platform': label,
     'MovieCount': count,
    })
platform

#  pie chart for distribution of movies on various platforms

In [ ]:
fig = px.pie(platform,names='Platform', values='MovieCount')
fig.update_traces(rotation=45, pull=[0.1,0.03,0.03,0.03,0.03],textinfo="percent+label", title='Movie Count per platform')
fig.show()

# Analysis of total count of movies in genres¶

In [ ]:
movies_data['Genres'].value_counts().head(5)

In [ ]:
top_5_genres = ['Drama','Documentary','Comedy', 'Comedy,Drama','Horror']
table = movies_data.loc[:,['Year','Genres','IMDb']]
table['AvgRating'] = table.groupby([table.Genres,table.Year])['IMDb'].transform('mean')
table.drop('IMDb', axis=1, inplace=True)
table = table[(table.Year>2009) & (table.Year<2020)]
table = table.loc[table['Genres'].isin(top_5_genres)]
table = table.sort_values('Year')

In [ ]:
fig=px.bar(table,x='Genres', y='AvgRating', animation_frame='Year', 
           animation_group='Genres', color='Genres', hover_name='Genres', range_y=[0,10])
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
top_movies=movies_datas[movies_datas['IMDb']==9.3][['Title','Directors']]
top_movies

In [ ]:
runtime_top_movies=movies_datas.loc[movies_datas['IMDb']==9.3][['Title','Runtime']]
runtime_top_movies

# pie chart for movie count for various countries

In [ ]:
movies_by_country = movies_data.groupby('Country')['Title'].count().reset_index().sort_values('Title',ascending = False).head(10).rename(columns = {'Title':'MovieCount'})
fig = px.pie(movies_by_country,names='Country', values='MovieCount')
fig.update_traces(rotation=180, pull=[0.1,0.03,0.03,0.03,0.03],textinfo="percent+label", title='Movie Count per Country')
fig.update_layout(showlegend=False)
fig.show()


# Lengthiest Movies according to IMDB runtime

In [ ]:
lengthiest_movie=movies_datas.sort_values(by='Runtime',ascending=False).head(10)

In [ ]:
fig=px.bar(lengthiest_movie,x='Title',y='IMDb',height=600,color='Title',range_y=[0,10])
fig.update_layout(title='IMDb Ratings Lengthiest Movie')
fig.show()

In [ ]:
movies_count_by_IMDB=movies_data.groupby('IMDb')['Title'].count().reset_index().sort_values('IMDb',ascending=False).head(10).rename(columns={'Title':'Count'})
fig=px.pie(movies_count_by_IMDB,names='IMDb',values='Count')
fig.update_traces(textinfo='percent+label',title='Movies Count based on IMDb Ratings')
fig.show()

# Top 5 movies on various platforms visualisation accordinng to imdb rating

In [ ]:
top5_netflix=movies_datas.loc[:,['Netflix','IMDb','Title','Year']]
top5_netflix=top5_netflix[top5_netflix['Netflix']==1]
top5_netflix=top5_netflix.sort_values('IMDb',ascending=False).head(5)
fig=px.bar(top5_netflix,x='Title',y='IMDb',height=600,color='IMDb',range_y=[0,10])
fig.update_layout(title='Top 5 Netflix Movies based on IMDb Rating')
fig.show()

In [ ]:
top5_hulu=movies_datas.loc[:,['Hulu','IMDb','Title','Year']]
top5_hulu=top5_hulu[top5_hulu['Hulu']==1]
top5_hulu=top5_hulu.sort_values('IMDb',ascending=False).head(5)
fig=px.bar(top5_hulu,x='Title',y='IMDb',height=600,color='Title',range_y=[0,10])
fig.update_layout(title='Top 5 Hulu Movies based on IMDb Rating')
fig.show()

In [ ]:
top5_prime=movies_datas.loc[:,['Prime Video','IMDb','Title','Year']]
top5_prime=top5_prime[top5_prime['Prime Video']==1]
top5_prime=top5_prime.sort_values('IMDb',ascending=False).head(5)
fig=px.bar(top5_prime,x='Title',y='IMDb',height=600,color='Title',range_y=[0,10])
fig.update_layout(title='Top 5 Prime Movies based on IMDb Rating')
fig.show()

In [ ]:
top5_disney=movies_datas.loc[:,['Disney+','IMDb','Title','Year']]
top5_disney=top5_disney[top5_disney['Disney+']==1]
top5_disney=top5_disney.sort_values('IMDb',ascending=False).head(5)
fig=px.bar(top5_disney,x='Title',y='IMDb',height=600,color='Title',range_y=[0,10])
fig.update_layout(title='Top 5 Disney Movies based on IMDb Rating')
fig.show()

In [ ]:
# Top 20 directors according to imdb rating


In [ ]:
#top20_dir=data.groupby(['Directors','IMDb'])['Title'].count().reset_index().sort_values('IMDb',ascending=False).head(20)
top20_dir=movies_data.loc[:,['Directors','IMDb','Title','Year']].sort_values('IMDb',ascending=False)

top20_dir.head(20)

# Analysis of ages of people on various platforms

In [ ]:
sum_age = movies_datas.groupby("Age").sum()
sum_age[["Netflix","Hulu","Prime Video","Disney+"]]

# number of movies produced each year per platform:

In [ ]:
# number of moive produce each year per platform:
sum_year = movies_datas.groupby("Year").sum()
sum_year[["Netflix","Hulu","Prime Video","Disney+"]]

In [ ]:
df_roto = movies_datas[movies_datas['Rotten Tomatoes'].notnull()]

In [ ]:
# convert str to float type for "Rotten Tomatoes" column:
df_roto['Rotten Tomatoes'] = df_roto['Rotten Tomatoes'].str.rstrip('%').astype('float')

In [ ]:
df_roto.head(10)

In [ ]:
# average IMDb rate on Disney+:
no_disney = df_roto['Disney+'].sum()
disney_avg_imdb = round(df_roto.loc[df_roto['Disney+']==1]['IMDb'].mean(),1)
# average IMDb rate on Hulu:
no_hulu = df_roto['Hulu'].sum()
hulu_avg_imdb = round(df_roto.loc[df_roto['Hulu']==1]['IMDb'].mean(),1)
# average IMDb rate on Netflix:
no_netflix = df_roto['Netflix'].sum()
netflix_avg_imdb = round(df_roto.loc[df_roto['Netflix']==1]['IMDb'].mean(),1)
# average IMDb rate on Prime Video:
no_prime = df_roto['Prime Video'].sum()
prime_avg_imdb = round(df_roto.loc[df_roto['Prime Video']==1]['IMDb'].mean(),1)

In [ ]:
# average Rotten Tomatoes rate on Disney+:
disney_avg_roto = round(df_roto.loc[df_roto['Disney+']==1]['Rotten Tomatoes'].mean(),1)
# average Rotten Tomatoes rate on Hulu:
hulu_avg_roto = round(df_roto.loc[df_roto['Hulu']==1]['Rotten Tomatoes'].mean(),1)
# average Rotten Tomatoes rate on Netflix:
netflix_avg_roto = round(df_roto.loc[df_roto['Netflix']==1]['Rotten Tomatoes'].mean(),1)
# average Rotten Tomatoes rate on Prime Video:
prime_avg_roto = round(df_roto.loc[df_roto['Prime Video']==1]['Rotten Tomatoes'].mean(),1)

In [ ]:
# create dataframe:
no_platform = (no_netflix,no_hulu,no_prime,no_disney)
col_names = ('Netflix','Hulu','Prime Video','Disney+')
avg_imdb = (netflix_avg_imdb,hulu_avg_imdb,prime_avg_imdb,disney_avg_imdb)
avg_roto = (netflix_avg_roto,hulu_avg_roto,prime_avg_roto,disney_avg_roto)
List = list(zip(col_names,no_platform,avg_imdb,avg_roto))
Counts =  pd.DataFrame(data=List,columns=['Platform','Number of Movie','Average IDMb rate','Average % Rotten Tomattoes rate'])
Counts

In [ ]:
px.bar(
       x=movies_data['Age'].value_counts().index,
       y=movies_data['Age'].value_counts().values,
       labels = {'x': "Age Restrictions", 'y':"Count"}
    )

In [ ]:
movies_datas.query('Netflix == 1 and Hulu == 1 and `Prime Video` == 1')['Title']

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.barplot(movies_datas.dropna(axis=0,subset=["IMDb"],inplace=False)["Year"],movies_data.dropna(axis=0,subset=["IMDb"],inplace=False)["IMDb"],ax=ax)

# correlation of movies according to imdb


In [ ]:
new_data=movies_datas.dropna(subset=["IMDb"])
new_data.corr()

# Visualing of distribution of movies on various platforms with boxplot

In [ ]:
list_platforms=["Netflix","Hulu","Prime Video","Disney+"]
fig,(ax1,ax2,ax3,ax4)=plt.subplots(1,4,sharey=True,figsize=(12,6))
fig.subplots_adjust(hspace=0, wspace=0)
ax1.boxplot(new_data["IMDb"][new_data["Netflix"]==1])
ax1.title.set_text("Netflix")
ax2.boxplot(new_data["IMDb"][new_data["Hulu"]==1])
ax2.title.set_text("Hulu")
ax3.boxplot(new_data["IMDb"][new_data["Prime Video"]==1])
ax3.title.set_text("Prime Video")
ax4.boxplot(new_data["IMDb"][new_data["Disney+"]==1])
ax4.title.set_text("Disney+")

# Getting unique values of genres,directors and countries.


In [ ]:
#Getting unique values of genres,directors and countries.
genres=movies_datas["Genres"].unique()
directors=movies_datas["Directors"].unique()
country=movies_datas['Country'].unique()
print(genres,directors,country)
#But as we can observe data set each of these have more than a single entity for almost each row . To make sure we don't repeat them in our unique value list we will 
#pick only different values out.
print(len(genres),len(directors),len(country))
final_genre=[]
for i in range(len(genres)):
    lis=list(map(str,genres[i].split(",")))
    for j in lis:
        if j not in final_genre:
            final_genre.append(j)

In [ ]:
final_genre

# Top movies produced each year according to imdb in various years

In [ ]:
movie_each_year= movies_datas.loc[movies_datas.groupby("Year")["IMDb"].idxmax()].reset_index()
movie_each_year.drop('index',axis=1,inplace=True)
movie_each_year.loc[:,['Title','Year','IMDb']].head(50)

# Change of best movie rating over the years

In [ ]:
#Change of best movie rating over the years
plt.figure(figsize=(10,5))
sns.lineplot(x='IMDb',y='Year',data=movie_each_year)

# Effect of runtime on rating

In [ ]:
#Effect of runtime on rating
plt.figure(figsize=(10,5))
sns.lineplot(x='Runtime',y='IMDb',data=movies_datas)

In [ ]:
movies_datas['Service Provider'] = movies_datas.loc[:,['Netflix','Prime Video','Disney+','Hulu']].idxmax(axis=1)
movies_datas.drop(['Netflix','Prime Video','Disney+','Hulu'],axis=1)

# Distribution of movie rating in each platform¶

In [ ]:
#Distribution of movie rating in each platform
plt.figure(figsize=(15,8))
sns.violinplot(x=movies_datas['IMDb'],y=movies_datas['Service Provider'])

# Average Runtime length of each ott platform


In [ ]:
#Average Runtime length of each ott platform
movies_datas.groupby("Service Provider")["Runtime"].mean()

# Average Runtime length of each ott platform

In [ ]:
#Average IMDb rating of each ott platform
movies_datas.groupby("Service Provider")["IMDb"].mean()

# Top 10 movies

In [ ]:
#Top 10 movies
movies_datas.sort_values('IMDb',ascending=False)[:10].reset_index().drop(['ID','index','Country','Language','Type','Rotten Tomatoes'],axis=1)

In [ ]:
top_rated_movies = movies_data.sort_values('IMDb',ascending = False).head(10)
fig = px.bar(top_rated_movies, x='Title', y='IMDb', color='IMDb', height=600)
fig.show()

In [ ]:
top_directors = movies_data.groupby('Directors')['Title'].count().reset_index().rename(columns = {'Title':'Movie Count'}).sort_values('Movie Count',ascending = False).head(10)
fig = px.bar(top_directors, x='Directors', y='Movie Count', color='Movie Count', height=600)
fig.show()


In [ ]:
#Top 10 movies of Disney
movies_datas[movies_datas['Service Provider']=='Disney+'].sort_values('IMDb',ascending=False)[:10].reset_index().drop(['ID','index','Country','Rotten Tomatoes','Language','Type'],axis=1)

In [ ]:
#Extracting the countries
country=[]
for i in range(len(movies_data)):
    x=movies_data['Country'][i]
    y=x.split(',')
    for j in range(len(y)):
        country.append(y[j])
        
country_c = Counter(country)
country_c_15=OrderedDict(country_c.most_common()[:15])

#Countries with most movies
data = [go.Bar(
   x = list(country_c_15.keys()),
   y = list(country_c_15.values())
)]
fig = go.Figure(data=data)
iplot(fig)

# Most of the movies have about 90-100 minutes of runtime. Runtime is positively skewed since there are some movies with very large runtime.

In [ ]:
#Most of the movies have about 90-100 minutes of runtime. Runtime is positively skewed since there are some movies with very large runtime.
sns.distplot(movies_datas['Runtime']);

# IMDb ratings of the movies roughly follows normal distribution with mean of about 6.5

In [ ]:
#IMDb ratings of the movies roughly follows normal distribution with mean of about 6.5
sns.distplot(movies_datas['IMDb']);

# Movies count by Language¶

In [ ]:
movie_count_by_language = movies_data.groupby('Language')['Title'].count().reset_index().sort_values('Title',ascending = False).head(10).rename(columns = {'Title':'Movie Count'})
fig = px.bar(movie_count_by_language, x='Language', y='Movie Count', color='Movie Count', height=600)
fig.show()

# Movies count by Language

In [ ]:
movies_by_country = movies_data.groupby('Country')['Title'].count().reset_index().sort_values('Title',ascending = False).head(10).rename(columns = {'Title':'Movie Count'})
fig = px.bar(movies_by_country, x='Country', y='Movie Count', color='Movie Count', height=600)
fig.show()

In [ ]:
top_genres = movies_data.groupby('Genres')['Title'].count().reset_index().rename(columns = {'Title':'Movie Count'}).sort_values('Movie Count',ascending = False).head(10)
fig = px.bar(top_genres, x='Genres', y='Movie Count', color='Movie Count', height=600)
fig.show()

In [ ]:
netflix_movies = movies_datas.loc[movies_data['Netflix'] == 1]
hulu_movies = movies_datas.loc[movies_data['Hulu'] == 1]
prime_video_movies = movies_datas.loc[movies_data['Prime Video'] == 1]
disney_movies = movies_data.loc[movies_data['Disney+'] == 1]

In [ ]:
netflix_movies.drop(['Hulu', 'Prime Video', 'Disney+', 'Type','Genres'], axis = 1)
hulu_movies.drop(['Netflix', 'Prime Video', 'Disney+', 'Type','Genres'], axis = 1)
prime_video_movies.drop(['Hulu', 'Netflix', 'Disney+', 'Type','Genres'], axis = 1)
disney_movies.drop(['Hulu', 'Prime Video', 'Netflix', 'Type','Genres'], axis = 1)

# visualising top 20 movies according to screenplay
 

In [ ]:
# Getting screenplay in hours
netflix_movies['screenplay'] = netflix_movies['Runtime']/60
hulu_movies['screenplay'] = hulu_movies['Runtime'] / 60
prime_video_movies['screenplay'] = prime_video_movies['Runtime'] / 60
disney_movies['screenplay'] = disney_movies['Runtime'] / 60


# Creating function to get screenplay of top 20 movies on desired platform
def top_20_sp(platform):
    top_20_screenplay=netflix_movies.sort_values(by='screenplay',ascending=False).head(20)
    plt.figure(figsize=(15,10))
    sns.barplot(x='screenplay',y='Title',data=top_20_screenplay,hue='Country')
    plt.legend(loc='lower right')
    plt.xlabel('Total hours')
    plt.ylabel('Movie')
    plt.title('Top 20 movies by screenplay')
    plt.show()
    
top_20_sp(netflix_movies)
top_20_sp(disney_movies)
top_20_sp(prime_video_movies)
top_20_sp(hulu_movies)

In [ ]:
n = 30

top_ratings = movies_data.sort_values(by="IMDb",ascending=False).reset_index().iloc[:n]

fig = px.bar(top_ratings,
             x = "Title", y='IMDb',
             color='IMDb',
             hover_data=["Directors","IMDb"],
             title=f"Top {n} best rated movies",
             height=1000,
             color_continuous_scale=px.colors.sequential.Viridis)
fig.show()

In [ ]:
#We often hear that nowadays they don´t make films as they used to... as if today´s films were not as good as the ones filmed years ago. Let´s see if that's entirely true...
df_year = movies_data.groupby("Year").mean()
df_year.drop(["Unnamed: 0","ID",'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type'],inplace=True,axis=1)
df_year["Movie Count"] = movies_data.groupby("Year").count()["ID"]
df_year.head()

In [ ]:
# Add a new column to our dataframe that indicates how many movies were made that year

# Get a dictionary from the df_year dataframe, where keys=year and values=movies made

d = df_year["Movie Count"].to_dict()
d.pop('IMDb', None)
d.pop("Runtime",None)

# Create our new column using the map function

movies_datas['Movie Count'] = movies_data['Year'].map(d)
movies_datas.head()

In [ ]:
#From now on we can add visual information to our plots that will remind us of how many movies were made on a certain year
x = movies_data["Year"]
y = movies_data["IMDb"]

plt.figure(figsize=(8,8))
sns.scatterplot(x,y,data=movies_datas,hue="Movie Count")

# Average movie ratings from 1902-2020


In [ ]:
Rating = df_year["IMDb"]
Year = df_year.index

fig = px.line(df_year, x=Year, y=Rating, 
              line_shape="spline", render_mode="svg",
              hover_name=movies_data.groupby("Year").mean().index,
              title="Average Movie Ratings 1902-2020")
fig.show()

In [ ]:
fig = px.box(movies_data, x="Year", y="IMDb",color="Year")
fig.show()

# scatter plot of movies based on runtime and according to imdb rating and year

In [ ]:
fig = px.scatter(movies_data,x="Runtime", y="IMDb",color="Year",
                 marginal_x="histogram",
                 marginal_y="histogram",
                 hover_name="Title",hover_data=["Age"])
fig.show()

In [ ]:
height = 800

fig2 = px.scatter(movies_data.dropna(),x="Runtime", y="IMDb",color="Age",
                marginal_x="histogram",
                hover_name="Title",hover_data=["Year"],
                height=height)

fig1 = px.scatter(movies_data.dropna(),x="Year", y="IMDb",color="Age",
                marginal_x="histogram",
                hover_name="Title",hover_data=["Runtime"],
                height=height)
fig1.show()
fig2.show()

In [ ]:
#Create a new dataframe specific for directors and the number of movies they directed

directors = pd.DataFrame(movies_data.groupby(["Directors"]).count()["ID"])
directors["No. of Films"] = directors["ID"]
directors.drop(["ID"],axis=1,inplace=True)

In [ ]:
n = 40
data = directors.sort_values(by="No. of Films",ascending=False).reset_index().iloc[:n]

fig = px.bar(data,
             x = "Directors", y='No. of Films',
             color='No. of Films',
             title=f"Top {n} directors with most directed movies")
fig.show()

# Directors who directed the best and worst IMDb ranked movies

In [ ]:
# Directors who directed the best and worst IMDb ranked movies

n = 30
x="Directors"

data1 = movies_data.groupby(by="Directors").mean().sort_values(by="IMDb",ascending=False).reset_index().iloc[:n]
data2 = movies_data.groupby(by="Directors").mean().sort_values(by="IMDb",ascending=True).reset_index().iloc[2:n]

# For the worst IMDb average we droped the worst two because they had an average of 0 and it´s not considered representative

fig = px.bar(data1,x=data1["Directors"],y=data1["IMDb"],color="IMDb",
             title=f"Top {n} Directors with the highest averaged movie ratings")
fig.show()

fig = px.bar(data2,x=data2["Directors"],y=data2["IMDb"],color="IMDb",
            title=f"Top {n} Directors with the lowest averaged movie ratings")
fig.show()

In [ ]:
# Directors who directed the longest and shortest movies

n = 30
x="Directors"

data1 = movies_data.groupby(by="Directors").mean().sort_values(by="Runtime",ascending=False).reset_index().iloc[:n]
data2 = movies_data.groupby(by="Directors").mean().sort_values(by="Runtime",ascending=True).reset_index().iloc[:n]

# For the worst IMDb average we droped the worst two because they had an average of 0 and it´s not considered representative

fig = px.bar(data1,x=data1["Directors"],y=data1["Runtime"],color="Runtime",
             title=f"Top {n} Directors with longest average runtime for directed movies")
fig.show()

fig = px.bar(data2,x=data2["Directors"],y=data2["Runtime"],color="Runtime",
            title=f"Top {n} Directors with shortest average runtime for directed movies")
fig.show()

In [ ]:
# Let´s turn this into a dataframe so that we can plot it

genres = pd.DataFrame.from_dict(d, orient='index').reset_index()
genres["Genre"] = genres["index"]
genres["Count"] = genres[0]
genres.drop(["index",0],axis=1,inplace=True)
genres.head()

In [ ]:
mixed_genres = []
for i in movies_data["Genres"].dropna():
    if "," in i:
        mixed_genres.append(i)

d = Counter(mixed_genres)

print(f"We found a total of {len(d)} different genres")

# Create dataframe

df_mixedg = pd.DataFrame.from_dict(d, orient='index').reset_index()
df_mixedg["Genre"] = df_mixedg["index"]
df_mixedg["Count"] = df_mixedg[0]
df_mixedg.drop(["index",0],axis=1,inplace=True)
df_mixedg.head()

In [ ]:
n = 30

# 3. Most common mixed genres.

fig = px.pie(df_mixedg.sort_values(by="Count",ascending=False).reset_index().iloc[:n], values='Count', names='Genre', 
             title=f"Top {n} most common movie mixed genres")
fig.show()

In [ ]:
all_genres = []

for genre in movies_data["Genres"].dropna():
    movie_genres = genre.split(",")
    for i in movie_genres:
        all_genres.append(i)

In [ ]:
df_genre_year = pd.DataFrame(index=range(1902,2021),columns=Counter(all_genres).keys())
#df_genre_year.astype("float64")
df_genre_year.head()

In [ ]:
for year in range(1902,2021):
    genre_year = []
    for genres in movies_data[movies_data["Year"]==year]["Genres"].dropna():
        for i in genres.split(","):
            genre_year.append(i)

    d1 = Counter(genre_year)
    
    if len(d)==0:
        #df_genre_year.loc[year].fillna(0,inplace=True)
        pass
    
    df_genre_year.loc[year] = pd.Series(d1)

# Check some random years 
    
df_genre_year.loc[1928:1933]

In [ ]:
df_genre_year.plot(figsize=(14,10))

In [ ]:
movies_data.hist(color='brown',figsize=(10,10))

In [ ]:
df_year = pd.DataFrame(movies_data.groupby(movies_data['Year']).Title.nunique())
df_year.nlargest(5,'Title')

# searching the movie on which platform it is present


In [ ]:
## Let's build a search function

def NextMovie(ScoreMovie,GenreMovie,YearMovie,LanguageMovie):
    possiblemovie1=movies_data.loc[movies_data['IMDb']>(ScoreMovie)]
    possiblemovie2=possiblemovie1.loc[movies_data['Genres'].str.contains(GenreMovie, na=False)]
    possiblemovie3=possiblemovie2.loc[movies_data['Year']>(YearMovie)]
    possiblemovie4=possiblemovie3.loc[movies_data['Language'].str.contains(LanguageMovie, na=False)]
    try:
      print(possiblemovie4['Title'])
    except:
      print("I\'m so sorry, no Title available, try to change parameters")

In [ ]:
#On Which Platform is this Movie Available?
def Availability(title):
    
    NetflixAvailable=Netflix.loc[Netflix['Title']==(title)]
    PrimeVideoAvailable=PrimeVideo.loc[PrimeVideo['Title']==(title)]
    DisneyAvailable=PrimeVideo.loc[PrimeVideo['Title']==(title)]
    HuluAvailable=PrimeVideo.loc[PrimeVideo['Title']==(title)]
   
        
    if (len(NetflixAvailable) >0):
            print('It is available on Netflix')
    else:
            print('It is not Available on Netflix')

    if (len(PrimeVideoAvailable) >0):
            print('It is available on PrimeVideo')
    else:
            print('It is not Available on PrimeVideo')

    if (len(DisneyAvailable) >0):
            print('It is available on Disney+')
    else:
            print('It is not Available on Disney+')

    if (len(HuluAvailable) >0):
            print('It is available on Hulu')
    else:
            print('It is not Available on Hulu')
        

In [ ]:
###Divide the DataSet according to the Platform
Netflix=movies_data[movies_data['Netflix']>0][['Title','Age','IMDb','Year','Directors','Genres','Country','Language','Runtime']]
PrimeVideo=movies_data[movies_data['Prime Video']>0][['Title','Age','IMDb','Year','Directors','Genres','Country','Language','Runtime']]
Hulu=movies_data[movies_data['Hulu']>0][['Title','Age','IMDb','Year','Directors','Genres','Country','Language','Runtime']]
Disney=movies_data[movies_data['Disney+']>0][['Title','Age','IMDb','Year','Directors','Genres','Country','Language','Runtime']]

In [ ]:
###Example
title=input("Enter the title")
Availability(title)

In [ ]:
movies_data = pd.melt(movies_data, id_vars = ['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Type', 'Directors', 'Genres', 'Country', 'Language'], 
        value_vars = ['Netflix', 'Hulu', 'Prime Video', 'Disney+'], var_name = 'Streaming Platform')
movies_data = movies_data[movies_data['value'] == 1].drop(['value', 'Type'], axis = 1) #Type is being dropped here because the value will always be 0, since the whole dataset represents only movies
movies_data.head()

In [ ]:
#Now, we have the data ready for some analysis, even though we can still do processing on the other rows, as Directors, Genres, Country and Language, but we will do that later.

#So, which platform has the most movies to offer?
movies_count = movies_data.groupby('Streaming Platform', as_index = False).count()
movies_count = movies_count[['Streaming Platform', 'ID']].rename({'ID' : 'Count'}, axis = 'columns')
movies_count = movies_count.sort_values(by = 'Count', ascending = False)
color_map={
    "Prime Video": "rgb(4, 165, 222)",
    "Netflix": "rgb(223, 9, 18)",
    "Hulu": "rgb(2, 228, 119)",
    "Disney+": "rgb(0, 0, 0)"
}

import plotly.express as px

fig = px.bar(movies_count, y='Streaming Platform', x="Count", color="Streaming Platform", orientation="h",
             color_discrete_map= color_map, text = "Count"
            )

fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title="Number of movies offered by platform",
      xaxis_tickfont_size=14,
    yaxis=dict(
        title='Streaming Platform',
        titlefont_size=16,
        tickfont_size=14
    ),
    legend=dict(
        x=1,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
)

fig.show()

In [ ]:
#In this criteria we can see that Prime Video blows the competition away with almost 3/4 of the total motives offered by the platforms, followed by Netflix. Let's see if quantity corresponds also to quality.
list_of_sp = movies_count['Streaming Platform'].tolist()

movies_df_not_null = movies_data[~movies_data['IMDb'].isnull()]

fig = go.Figure()

rows_counter = 0
for sp, clr in zip(list_of_sp, color_map.values()):
        
        fig.add_trace(go.Box(
            x=movies_df_not_null[movies_df_not_null['Streaming Platform'] == sp]['Streaming Platform'],
            y=movies_df_not_null[movies_df_not_null['Streaming Platform'] == sp]['IMDb'],
            name=sp,
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=clr,
            marker_size=2,
            line_width=1)
        )
        
        min_value = round(np.percentile(movies_df_not_null[movies_df_not_null['Streaming Platform'] == sp]['IMDb'], 0), 2)
        first_quartile = round(np.percentile(movies_df_not_null[movies_df_not_null['Streaming Platform'] == sp]['IMDb'], 25), 2)
        median = round(np.percentile(movies_df_not_null[movies_df_not_null['Streaming Platform'] == sp]['IMDb'], 50), 2)
        third_quartile = round(np.percentile(movies_df_not_null[movies_df_not_null['Streaming Platform'] == sp]['IMDb'], 75), 2)
        max_value = round(np.percentile(movies_df_not_null[movies_df_not_null['Streaming Platform'] == sp]['IMDb'], 100), 2)
        
        for y_desc, y_value in zip(('MinV: ', 'Q1: ', 'Med: ', 'Q3: ', 'MaxV: '), (min_value, first_quartile, median, third_quartile, max_value)):
        
            fig.add_annotation(
                        x=rows_counter + 0.35,
                        ax = 0,
                        ay = 0,
                        y=y_value,
                        text=y_desc + str(y_value))

        rows_counter += 1
        
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title='Distribution of movies IMDb score by streaming platform',
    yaxis=dict(
        title='IMDb score',
        titlefont_size=16,
        tickfont_size=14
    ))

        
fig.show()

In [ ]:
#Even though the data did not show that much of a discrepancy between the platforms as the last analysis, now Disney+ emerged as victorious, with a better distribution, in general, of IMDb scores for its movies, followed by Netflix. But, let's say you are not so much of a hard user and will only watch 1 movie per week, resulting in 52 movies per year. Which platform has the best 52 movies?
movies_data.sort_values(by = 'IMDb', ascending = False, inplace = True)
results_list = {}
for sp in movies_count['Streaming Platform'].tolist():
    sp_best_df = movies_data[movies_data['Streaming Platform'] == sp].head(n = 52)
    IMDb_mean = round(sp_best_df['IMDb'].mean(), 2)
    results_list[sp] = IMDb_mean
best_movies_df = pd.DataFrame.from_dict(results_list, orient = 'index', columns = ['AVG Score']).sort_values(by = 'AVG Score', ascending = True)


In [ ]:
fig = px.bar(best_movies_df, y=best_movies_df.index, x="AVG Score", color=best_movies_df.index, orientation="h",
             color_discrete_map = color_map, text = "AVG Score"
            )

fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title="Average score of the best 52 movies by platform",
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Streaming Platform',
        titlefont_size=16,
        tickfont_size=14
    ),
    xaxis = dict(
        range = [best_movies_df['AVG Score'].min() - 0.1, best_movies_df['AVG Score'].max() + 0.1],
        title = "Average Score"
    ),
    legend=dict(
        x=1,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)',
        title = 'Streaming Platform'
    ),
)

fig.show()

In [ ]:
#Since we are working with a small number of movies, we can work with the average, since no outliers will mess up the numbers. Now we see a different pattern, similar to the number of movies offered by platform. Prime Video leading with Netflix right behing, and Hulu and Disney+ almost drawn. Now, let's see how are genres distributed. First, let's get the data in a easier way do deal with.
genres_split = movies_data['Genres'].str.get_dummies(',')
genres_movie_df = pd.concat([movies_data, genres_split], axis = 1)
genres_movie_df = pd.melt(genres_movie_df, id_vars = ['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Directors', 'Genres', 'Country', 'Language', 'Streaming Platform'], 
        value_vars = genres_split.columns, var_name = 'Genre')
genres_movie_df = genres_movie_df[genres_movie_df['value'] == 1].drop(['value', 'Genres'], axis = 1)
genres_movie_df.head()

In [ ]:
genres_count = genres_movie_df.groupby('Genre', as_index = False).count()
genres_count = genres_count[['Genre', 'ID']].rename({'ID' : 'Count'}, axis = 'columns')
genres_count = genres_count.sort_values(by = 'Count', ascending = False)

import plotly.express as px

fig = px.bar(genres_count.head(n = 15), y='Genre', x="Count", color='Genre', orientation="h", text = "Count"
            )

fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title="Number of movies segmented by genre",
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Movie genre',
        titlefont_size=16,
        tickfont_size=14
    ),
    legend=dict(
        x=1,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
         bordercolor='rgba(255, 255, 255, 0)'
    ),
)

fig.show()

In [ ]:
countries_split = movies_data['Country'].str.get_dummies(',')
countries_movie_df = pd.concat([movies_data, countries_split], axis = 1)
countries_movie_df = pd.melt(countries_movie_df, id_vars = ['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Directors', 'Genres', 'Language', 'Streaming Platform'], 
        value_vars = countries_split.columns, var_name = 'Country')
countries_movie_df = countries_movie_df[countries_movie_df['value'] == 1].drop(['value'], axis = 1)
countries_movie_df
    

In [ ]:
countries_count = countries_movie_df.groupby('Country', as_index = False).count()
countries_count = countries_count[['Country', 'ID']].rename({'ID' : 'Count'}, axis = 'columns')
countries_count = countries_count.sort_values(by = 'Count', ascending = False)

import plotly.express as px

fig = px.bar(countries_count.head(n = 15), y='Country', x="Count", color='Country', orientation="h", text = "Count"
            )

fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title="Number of movies segmented by country",
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Movie genre',
        titlefont_size=16,
        tickfont_size=14
    ),
    legend=dict(
        x=1,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
)
fig.show()

In [ ]:
sns.kdeplot(movies_data['IMDb'],shade = True)
plt.axvline(x = movies_data['IMDb'].median(),color = 'red',label = 'median')
plt.legend()

In [ ]:
Lang = movies_datas.drop('Language', axis=1).join(
    movies_data['Language'].str.split(',', expand=True).stack().reset_index(drop=True, level=1).rename('Language'))

Lang_count = Lang['Language'].value_counts().head(25).reset_index().set_index('index')


In [ ]:
L_ratings = Lang.groupby('Language')['IMDb'].median()
L_ratings = L_ratings.reset_index().set_index('Language')

In [ ]:
Top_10_lang = L_ratings.loc[['English','Hindi','Spanish','French','German','Italian'
                                                      ,'Japanese','Korean','Mandarin','Russian'],'IMDb']
Top_10_lang

In [ ]:
Top_10_lang = Top_10_lang.reset_index().set_index('Language')
English = Lang.loc[Lang['Language']=='English','IMDb'].reset_index().set_index('index')

In [ ]:
fig,axes = plt.subplots(1,2,figsize = (18,6))

sns.kdeplot(English['IMDb'],ax = axes[1],shade = True)
plt.axvline(English['IMDb'].median(),color = 'red')

sns.barplot(x=Top_10_lang.index,y=Top_10_lang['IMDb'],ax = axes[0])
plt.show()

In [ ]:
L_Netflix = Lang.loc[Lang['Netflix'] == 1,'Language'].value_counts().reset_index().set_index('index')
L_Prime =  Lang.loc[Lang['Prime Video'] == 1,'Language'].value_counts().reset_index().set_index('index')
L_Hulu = Lang.loc[Lang['Hulu'] == 1,'Language'].value_counts().reset_index().set_index('index')
L_Disney = Lang.loc[Lang['Disney+'] == 1,'Language'].value_counts().reset_index().set_index('index')

In [ ]:
fig , axes = plt.subplots(2,2,figsize = (12,12))
 
plt.subplots_adjust(hspace = 0.6,wspace = 0.5)    
    
L_Netflix.head(10).plot(kind = 'bar',ax = axes[0,0])
axes[0,0].set_title('Netflix')
axes[0,0].set_xlabel('')
axes[0,0].set_ylabel('')

L_Prime.head(10).plot(kind = 'bar',ax = axes[0,1])
axes[0,1].set_title('Prime Video')
axes[0,1].set_xlabel('')
axes[0,1].set_ylabel('')


L_Hulu.head(10).plot(kind = 'bar',ax = axes[1,0])
axes[1,0].set_title('Hulu')
axes[1,0].set_xlabel('')
axes[1,0].set_ylabel('')

L_Disney.head(10).plot(kind = 'bar',ax = axes[1,1])
axes[1,1].set_title('Disney')

axes[1,1].set_xlabel('')
axes[1,1].set_ylabel('')

fig.text(0.5, 0.004, 'Language', ha='center',fontsize = 'large')
fig.text(0.004, 0.5, 'Count', va='center', rotation='vertical',fontsize = 'large')
plt.show()



In [ ]:
Genre = movies_datas.drop('Genres', axis=1).join(
    movies_datas['Genres'].str.split(',', expand=True).stack().reset_index(drop=True, level=1).rename('Genre'))
Genre_count = Genre['Genre'].value_counts().reset_index().set_index('index')

In [ ]:
G_Netflix = Genre.loc[Genre['Netflix'] == 1,'Genre'].value_counts().reset_index().set_index('index')
G_Prime =  Genre.loc[Genre['Prime Video'] == 1,'Genre'].value_counts().reset_index().set_index('index')
G_Hulu = Genre.loc[Genre['Hulu'] == 1,'Genre'].value_counts().reset_index().set_index('index')
G_Disney = Genre.loc[Genre['Disney+'] == 1,'Genre'].value_counts().reset_index().set_index('index')

In [ ]:
fig , axes = plt.subplots(2,2,figsize = (12,12))

 
G_Netflix.head(10).plot(kind = 'bar',ax = axes[0,0],color = 'brown')
axes[0,0].set_title('Netflix')
axes[0,0].set_xlabel('')
axes[0,0].set_ylabel('')

G_Prime.head(10).plot(kind = 'bar',ax = axes[0,1],color = 'green')
axes[0,1].set_title('Prime Video')
axes[0,1].set_xlabel('')

G_Hulu.head(10).plot(kind = 'bar',ax = axes[1,0],color = 'gray')
axes[1,0].set_title('Hulu')
axes[1,0].set_xlabel('')
axes[1,0].set_ylabel('')


G_Disney.head(10).plot(kind = 'bar',ax = axes[1,1])
axes[1,1].set_title('Disney')
axes[1,1].set_xlabel('')

plt.tight_layout()
fig.text(0.5, 0.004, 'Genre', ha='center',fontsize = 'large')
fig.text(0.004, 0.5, 'Count', va='center', rotation='vertical',fontsize = 'large')
plt.show()

# conclusion
From the dataset it can be concluded that the distribution of movies on ott were on prime apart from other ott platforms and we were able to analyse the total number of movies and top 20 directors and top rated movies and we were able to develop a recommender system based on actors and genre and imdb rating

# Please upvote if you liked my work****